# Batch processing of large data sets

For the analysis of large data sets, we recommend parallel batch processing for faster run times. Below are scripts for structural and functional analysis using of multiple tif-files using `multiprocessing`. 

## Structural analysis

Parameters for batch analysis should be optimized on one or few representative tif-files. Details on the analysis of sarcomere structure for a single tif-file can be found [here](tutorial_structure_analysis.ipynb).

## Motion analysis

Parameters for batch analysis should be optimized on one or few representative tif-files. Details on the functional analysis of a single movie of a beating cardiomyocyte can be found [here](tutorial_motion_analysis.ipynb).